In [3]:
#Andrej karpathy -- Assignments 

#Ex-1
What i have to do is:
    
#training a trigram model, 2 char as input to predict 3rd one. 
# I am using both counting and neural network
# I have to evaulate the loss
# And answer the question, did it improve over the bigram model? (he made video about the bigram model)


In [4]:
# I have to make 2d matrix of size (27*27)(rows) *(27)(columns)
# because .a,.b ..... aa,ab ........, za,zb..... is (27*27)
# and output can be any 27 letters

#ok let's start


In [5]:
words = open('names.txt','r').read().splitlines()

In [6]:
words[:10]

['emma',
 'olivia',
 'ava',
 'isabella',
 'sophia',
 'charlotte',
 'mia',
 'amelia',
 'harper',
 'evelyn']

In [7]:
import torch

In [8]:
N = torch.zeros((27*27,27),dtype = torch.int32)

In [9]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}

In [10]:
for w in words:
  chs = ['.'] + list(w) + ['.']
  for ch1,ch2,ch3 in zip(chs,chs[1:],chs[2:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    ix3 = stoi[ch3]
    N[(27*ix1)+(ix2),ix3] +=1
# mx = -1
# for i in range(27*27):
#     for j in range(27):
#         mx = max(mx,N[i,j])
# print(mx)

In [29]:
#let's visualize, i know it's not a good considering how big the picture would be, hehehe

import matplotlib.pyplot as plt
%matplotlib inline

# plt.figure(figsize=(16*27,16))
# plt.imshow(N, cmap='Blues')
# for i in range(27*27):
#     for j in range(27):
        
#         first_char = i//27
#         second_char = i%27
        
#         chstr = itos[first_char] + itos[second_char] + '-'+ itos[j]
#         plt.text(j, i, chstr, ha="center", va="bottom", color='gray')
#         plt.text(j, i, N[i, j].item(), ha="center", va="top", color='gray')
# plt.axis('off');

# i don't know why it is taking so long, let's do it later

In [12]:
print(N[2])  # .b paxi kun char ko count badi xa?

tensor([  0, 169,   0,   0,   0, 253,   0,   0,   9,  41,   1,   0,  85,   0,
          0,  77,   0,   0, 646,   0,   0,  21,   0,   0,   0,   4,   0],
       dtype=torch.int32)


In [13]:
P = (N+1).float()
P /= P.sum(1,keepdim = True) #729 * 27 / 1 * 27  broadcast able
print(P[0].size())
print(P[0].sum())

torch.Size([27])
tensor(1.)


In [14]:
#nice, 1 nai aauna parne ho


#let's generate a name now. hehe

g = torch.Generator().manual_seed(2147483647) #using this generator

for i in range(10): #let's generate 10 names

  out = []
  ix = 0 # zero index ma hereko._ kun letter chain first ma suru hunxa
  #first i have to input two letters . and any 26 char randomly:

  gt = torch.randint(0,27,(1,)).item()  
    
  while True:
    # p = N[ix].float()  #sab ko number nikalyo ix th char paxi k aauxa vanera
    # p = p / p.sum() # probabilty ma lageko
    
    ix = (ix*27) + (gt)   # giving 2 char, because this is trigram model
    
    
    p = P[ix]
    out.append(itos[gt])
    ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
    #sample 1 leko. jun chain ix th char ko paxadi aaune probability badi xa, tei char ko index ho yo new ix
    ix , gt = gt, ix

    if gt == 0:  #end vaye ta break garnu paryo ni ta
      break
  print(''.join(out))

suuwjde
vianasid
gus
elay
zain
na
devin
itoper
grayel
sus


In [15]:
# wow i got the name rock. haha. 
# novin, could be a good AI name.
# haree, it's literally my brother name
# wow, nice


In [16]:
# Analyzing above result, i think trigram is better than bigram, but not sure
# I have some flaw in my implemention.  i guess randomly the first char. but i could choose those charters which are likely to come first
# MY way would generate a name even more unique so i am ok with it.

In [17]:
#if we consider all letters are equally likely then 1/27 ~ 4%
# so any probabilty > 4% is better


In [18]:
#finding loss


log_likelihood = 0.0

n = 0
for w in words:
  chs = ['.'] + list(w) + ['.']
  for ch1,ch2,ch3 in zip(chs,chs[1:],chs[2:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    ix3 = stoi[ch3]
    prob = P[(ix1*27) + ix2, ix3]
    log_likelihood += torch.log(prob)
    n+=1
    # print(f"{ch1}{ch2}:  {prob:.4f}")

print(f"loss: {log_likelihood=}")
nll  = - log_likelihood
print(f"negative log likelihood: {nll=}")
print(f"Avg negative-log likelihood: {nll/n=}")

loss: log_likelihood=tensor(-410414.9688)
negative log likelihood: nll=tensor(410414.9688)
Avg negative-log likelihood: nll/n=tensor(2.0927)


In [19]:
# Avg negative log likelihood of our trigram model is less than bigram model hence this is better
# i still don't understand how loss works in this case?
# is predicting next word based on more past words makes our prediction better?
# How this loss is finding this model is better?
# i am still confused


In [20]:
## Now let's use neural network let's go......
#creating training set

In [38]:

xs, ys = [], []
for w in words[:1]:
  chs = ['.'] + list(w) + ['.']
  for ch1,ch2,ch3 in zip(chs,chs[1:],chs[2:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    ix3 = stoi[ch3]
    xs.append((ix1*27)+ix2)
    ys.append(ix3)
    print(f"{ch1+ch2} {ch3}")
    
xs = torch.tensor(xs)
ys = torch.tensor(ys)
print(xs)
print(ys)

.e m
em m
mm a
ma .
tensor([  5, 148, 364, 352])
tensor([13, 13,  1,  0])


In [26]:
#one hot encodeing is needed. omg vector of size (27*27) is this efficient?

In [27]:
import torch.nn.functional as F
xenc = F.one_hot(xs, num_classes = (27*27)).float()
xenc.shape

torch.Size([4, 729])

In [33]:
# plt.imshow(xenc)

In [34]:
xenc.dtype

torch.float32

In [35]:
W = torch.randn((27*27,27))

In [45]:
print((xenc@W)[2,13])
(W[:,13]).shape

tensor(-0.8599)


torch.Size([729])

In [41]:
#same
print(xenc[2]@W[:,13])  # still

tensor(-0.8599)


In [46]:
# ok neural network architecture looks like this
# 27*27 neurons in the input ---->  27 outputs..    NO hidden layers

# now we have to introduce loss to change the weights.
# multiple examples line, label vai halxa ani gradient line ani parameters update garne.

# kaile a paxi b aaule kaile a paxi c aauxla kaile feri a paxi b aaula. tara hamro large examples ko loss nikalera
# parameters update garda, milla. a paxi b aaune probabilty ali badi hola tei ho


In [47]:
logits = xenc @ W  #log counts

#yo logits ta ouput ho hai? yeslai softmax garda hola
counts = logits.exp() # equivalent to N
probs = counts/counts.sum(1,keepdim = True)

probs[0].sum()

#soft-max use gareko hai

tensor(1.0000)

In [59]:
nlls = torch.zeros(4)
for i in range(4):
    
  # i-th bigram:
  x = xs[i].item() # input character index
  y = ys[i].item() # label character index
  
  ch1 = (x//27)
  ch2 = x%27
  print('--------')
  print(f'bigram example {i+1}: {itos[ch1]+itos[ch2]}{itos[y]} (indexes {x},{y})')
  print('input to the neural net:', x)
  print('output probabilities from the neural net:', probs[i])
  print('label (actual next character):', y)
  p = probs[i, y]
  print('probability assigned by the net to the the correct character:', p.item())
  logp = torch.log(p)
  print('log likelihood:', logp.item())
  nll = -logp
  print('negative log likelihood:', nll.item())
  nlls[i] = nll


print('=========')
print('average negative log likelihood, i.e. loss =', nlls.mean().item())

--------
bigram example 1: .em (indexes 5,13)
input to the neural net: 5
output probabilities from the neural net: tensor([0.0777, 0.0145, 0.0035, 0.0203, 0.0753, 0.0746, 0.0145, 0.0253, 0.0080,
        0.0057, 0.0010, 0.0357, 0.0516, 0.0286, 0.0225, 0.0042, 0.0639, 0.0066,
        0.0455, 0.0768, 0.0063, 0.0490, 0.0575, 0.0340, 0.0058, 0.1690, 0.0224])
label (actual next character): 13
probability assigned by the net to the the correct character: 0.02863534353673458
log likelihood: -3.5531134605407715
negative log likelihood: 3.5531134605407715
--------
bigram example 2: emm (indexes 148,13)
input to the neural net: 148
output probabilities from the neural net: tensor([0.0094, 0.0215, 0.0240, 0.0147, 0.0182, 0.0547, 0.0531, 0.0396, 0.0308,
        0.0081, 0.0966, 0.0236, 0.0237, 0.0233, 0.0143, 0.0625, 0.0812, 0.0274,
        0.0160, 0.0347, 0.0249, 0.0064, 0.0426, 0.0391, 0.1395, 0.0622, 0.0080])
label (actual next character): 13
probability assigned by the net to the the correct cha

In [61]:
#loss ramro xaina, let's train, let's goooo

In [62]:
# (this may be wrong, I just gussed based on  what i have understood)

# Q. what is mean by training here?
# -> it is updating our W matirx.
# Q. how you update our W matrix here?
# -> traing example banako xa ni ta. so hamle forward pass garxum. our model perform based on it's parameter
#    now we find the loss. (using our training data as label, ofcourse). 
#    sabai data ma ekai choti garne hola hai? train?
#    ani feri gareko garekai kina garne ra?
#    gradient descent vako xani ta so. loop gareko garekai garesi local minima ma janxa ni ta.

# nice

In [63]:
#gradient descent let's goooooooooooooooooooooooo


In [80]:
# create the dataset

xs, ys = [], []
for w in words:
  chs = ['.'] + list(w) + ['.']
  for ch1,ch2,ch3 in zip(chs,chs[1:],chs[2:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    ix3 = stoi[ch3]
    xs.append((ix1*27)+ix2)
    ys.append(ix3)
    
xs = torch.tensor(xs)
ys = torch.tensor(ys)
num = xs.nelement()
print('number of examples: ', num)

# initialize the 'network'
g = torch.Generator().manual_seed(2147483647)
W = torch.randn((27*27, 27), generator=g, requires_grad=True)

number of examples:  196113


In [82]:
for k in range(100):
    
    
    #forward pass
    
    xenc = F.one_hot(xs, num_classes = (27*27)).float() #input to the network: one-hot encoding
    logits = xenc@W  #log-counts predict gareko hai
    counts = logits.exp()  #coutns, equivalent to N
    probs = counts/counts.sum(1,keepdim = True) #probabilities for next character
    
    loss = -probs[torch.arange(num),ys].log().mean()  + 0.01 * (W**2).mean()  # i haven't understood this second part
    
    #maile bujeko yo loss chai,
#     probs[torch.arrange(num),ys] this gives probability predcited by our model for ground truth. if it is less than 
#     100% model we give it a loss i.e -loglikelihood now model updates it's parameters.
    
#     Q. I am super-amazed how this loss worked? 
#     --> because we are not always sure what letters comes after pairs in real names. any letter can come. 
#         but some are more likely.
#     --> i think our model will twiks our W such that it gives more probabilty for more likely words based on training set
    
#     Q. Still confused how loss will converged? I suspect it will not. (noob me, think later)
#     Q. 

     
    print(loss.item())
    #backward pass:
    
    W.grad = None # paile sabaile gradient lai zero gareko
    loss.backward()  #magic hehe
    
    #update
    
    W.data += -50*W.grad   #remember to increment opp to direction of gradient, otherwise face the consequences)

    

2.409698009490967
2.4073662757873535
2.405068874359131
2.4028048515319824
2.400573492050171
2.398374080657959
2.3962056636810303
2.3940680027008057
2.391960382461548
2.3898818492889404
2.3878324031829834
2.3858110904693604
2.383816719055176
2.381850004196167
2.3799092769622803
2.3779945373535156
2.376105546951294
2.3742408752441406
2.372401237487793
2.3705852031707764
2.36879301071167
2.367023468017578
2.365276575088501
2.3635518550872803
2.361848831176758
2.3601672649383545
2.358506441116333
2.3568663597106934
2.355246067047119
2.3536458015441895
2.352065324783325
2.35050368309021
2.3489606380462646
2.3474361896514893
2.3459293842315674
2.3444411754608154
2.3429694175720215
2.341515302658081
2.340078115463257
2.3386569023132324
2.337252378463745
2.3358638286590576
2.3344905376434326
2.3331332206726074
2.3317906856536865
2.330463171005249
2.3291501998901367
2.3278512954711914
2.3265669345855713
2.325296640396118
2.3240394592285156
2.322796106338501
2.321565628051758
2.3203482627868652


In [ ]:
#wow this is giving much better results

In [93]:
# finally, sample from the 'neural net' model
g = torch.Generator().manual_seed(2147483647)

for i in range(10):

  out = []
  ix = 0
  gt = torch.randint(1,27,(1,)).item()  
  while True:

    # ----------
    # BEFORE:
    #p = P[ix]
    # ----------
    # NOW:
    ix = (ix*27) + (gt)
    out.append(itos[gt])
    xenc = F.one_hot(torch.tensor([ix]), num_classes=27*27).float()
    logits = xenc @ W # predict log-counts
    counts = logits.exp() # counts, equivalent to N
    p = counts / counts.sum(1, keepdims=True) # probabilities for next character
    # ----------

    ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
    ix , gt = gt, ix
    
    if gt == 0:
      break
  print(''.join(out))

el
kojde
milyaqah
gus
elay
lein
gandi
xrvtonian
caree
aviahnia


In [ ]:
# Different name came, i don't know why?
# some names doesn't seems right

